In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.util import Surv
from lifelines.utils import concordance_index
from sklearn.metrics import roc_auc_score

In [14]:


# Load data
# df = pd.read_csv('./data/equity-post-HCT-survival-predictions/train.csv')


def prepare_data(df, categorical_cols, id_col='ID'):
    # Create a copy of the dataframe
    data = df.copy()
    
    # Ensure efs is integer (event indicator: 0 or 1)
    data['efs'] = data['efs'].astype(int)
    
    # Drop the ID column if it exists
    if id_col in data.columns:
        data = data.drop(columns=[id_col])
        print(f"Dropped column: {id_col}")
    else:
        print(f"No column named '{id_col}' found in the dataset")
    
    # Separate features and target
    X = data.drop(['efs', 'efs_time'], axis=1)
    y = Surv.from_arrays(event=data['efs'], time=data['efs_time'])
    
    
    
    # Define preprocessing for categorical and numerical columns
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])
    
    numerical_cols = [col for col in X.columns if col not in categorical_cols]
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median'))
    ])
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_cols),
            ('num', numerical_transformer, numerical_cols)
        ])
    
    # Fit and transform the data
    X_preprocessed = preprocessor.fit_transform(X)
    
    # Get feature names after one-hot encoding
    cat_feature_names = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_cols)
    feature_names = np.concatenate([cat_feature_names, numerical_cols])
    
    return X_preprocessed, y, feature_names, preprocessor

def split_train_and_evaluate_with_sksurv(df, train_size=0.7, val_size=0.15, test_size=0.15,
                                         categorical_cols=[
        'dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'tbi_status',
        'arrhythmia', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe',
        'prim_disease_hct', 'cmv_status', 'tce_imm_match', 'rituximab',
        'prod_type', 'cyto_score_detail', 'conditioning_intensity', 'ethnicity',
        'obesity', 'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hepatic_severe',
        'prior_tumor', 'peptic_ulcer', 'gvhd_proph', 'rheum_issue', 'sex_match',
        'race_group', 'hepatic_mild', 'tce_div_match', 'donor_related',
        'melphalan_dose', 'cardiac', 'pulm_moderate'
    ], id_col='ID'):
    assert train_size + val_size + test_size == 1.0, "Split sizes must sum to 1"
    
    # Prepare data
    X, y, feature_names, preprocessor = prepare_data(df, categorical_cols, id_col)
    print(feature_names)
    
    # Split into train + (val + test)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42
    )

    
    # Print sizes
    print(f"Training set size: {len(X_train)} ({len(X_train)/len(X):.2%})")
    print(f"Validation set size: {len(X_val)} ({len(X_val)/len(X):.2%})")
    print(f"Test set size: {len(X_test)} ({len(X_test)/len(X):.2%})")
    
    # {'n_estimators': 300, 'learning_rate': 0.033266652862807916, 'max_depth': 10, 'min_samples_split': 7, 'subsample': 0.9566352087990605, 'max_features': 'log2', 'n_iter_no_change': 8, 'validation_fraction': 0.17499766883832907}
    
    # Define and train the model
    model = GradientBoostingSurvivalAnalysis(
        n_estimators=300,
        learning_rate=0.033266652862807916,
        max_depth=10,
        min_samples_split=7,
        subsample=0.9566352087990605,
        random_state=42,
        max_features='log2', # 'sqrt', 0.3-0.7, 
        n_iter_no_change=8, # Set to 10, 20, or 50, and pair with a validation fraction
        validation_fraction=0.17499766883832907, # 0.1–0.3
        verbose=1
    )
    
    model.fit(X_train, y_train)
    
    # Predict risk scores
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    
    # Extract event times and indicators for evaluation
    t_train = y_train['time']
    e_train = y_train['event']

    t_test = y_test['time']
    e_test = y_test['event']
    
    print("Test prediction head: ")
    print(test_pred[:5])

    
    # Calculate C-index
    c_index_train = concordance_index(t_train, train_pred, e_train)
    c_index_test = concordance_index(t_test, test_pred, e_test)
    print(f"\nTraining C-index: {c_index_train:.4f}")
    print(f"Test C-index: {c_index_test:.4f}")
    
    
    # Feature importance
    importance = pd.DataFrame({
        'feature': feature_names,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\nTop 10 most important features:")
    print(importance.head(10))
    
    return model, preprocessor, X_train, X_val, X_test, y_train, y_val, y_test, val_pred, test_pred

# Load and evaluate the model
df = pd.read_csv('./data/equity-post-HCT-survival-predictions/train.csv')
# df['conditioning_intensity'] = df['conditioning_intensity'].fillna("TBD")

# Call the function, specifying the ID column to remove
model, preprocessor, X_train, X_val, X_test, y_train, y_val, y_test, val_pred, test_pred = split_train_and_evaluate_with_sksurv(
    df,
    train_size=0.75,
    val_size=0.10,
    test_size=0.15,
    id_col='ID'  # Adjust this to match your actual ID column name
)

Dropped column: ID
['dri_score_High' 'dri_score_High - TED AML case <missing cytogenetics'
 'dri_score_Intermediate'
 'dri_score_Intermediate - TED AML case <missing cytogenetics'
 'dri_score_Low' 'dri_score_Missing' 'dri_score_Missing disease status'
 'dri_score_N/A - disease not classifiable'
 'dri_score_N/A - non-malignant indication' 'dri_score_N/A - pediatric'
 'dri_score_TBD cytogenetics' 'dri_score_Very high'
 'psych_disturb_Missing' 'psych_disturb_No' 'psych_disturb_Not done'
 'psych_disturb_Yes' 'cyto_score_Favorable' 'cyto_score_Intermediate'
 'cyto_score_Missing' 'cyto_score_Normal' 'cyto_score_Not tested'
 'cyto_score_Other' 'cyto_score_Poor' 'cyto_score_TBD' 'diabetes_Missing'
 'diabetes_No' 'diabetes_Not done' 'diabetes_Yes' 'tbi_status_No TBI'
 'tbi_status_TBI + Cy +- Other'
 'tbi_status_TBI +- Other, -cGy, fractionated'
 'tbi_status_TBI +- Other, -cGy, single'
 'tbi_status_TBI +- Other, -cGy, unknown dose'
 'tbi_status_TBI +- Other, <=cGy' 'tbi_status_TBI +- Other, >cGy

KeyboardInterrupt: 

In [12]:
X_train.columns

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [5]:
# Load and evaluate the model
df = pd.read_csv('./data/equity-post-HCT-survival-predictions/train.csv')

# Define the mapping for the bins
dri_bins = {
    'High': ['High'],
    'Medium': ['Intermediate', 'High - TED AML case <missing cytogenetics', 
               'Intermediate - TED AML case <missing cytogenetics', 'Low', 
               'Missing disease status'],
    'Low': ['N/A - disease not classifiable', 'N/A - non-malignant indication', 
            'N/A - pediatric', 'TBD cytogenetics', 'Very high']
}

# Function to map dri_score to new bins
def bin_dri_score(score):
    if pd.isna(score):  # Handle NaN values
        return 'Low'  # Assuming NaN goes to 'Low', adjust if needed
    for bin_name, values in dri_bins.items():
        if score in values:
            return bin_name
    return 'Low'  # Default for any unmapped values (e.g., edge cases)


df['dri_score'] = df['dri_score'].apply(bin_dri_score)



# Call the function, specifying the ID column to remove
model, preprocessor, X_train, X_val, X_test, y_train, y_val, y_test = split_train_and_evaluate_with_sksurv(
    df,
    train_size=0.7,
    val_size=0.15,
    test_size=0.15,
    id_col='ID'  # Adjust this to match your actual ID column name
)

Dropped column: ID
Training set size: 20160 (70.00%)
Validation set size: 4320 (15.00%)
Test set size: 4320 (15.00%)
      Iter       Train Loss      OOB Improve   Remaining Time 
         1       62287.2491          16.7188           11.12m
         2       62099.0223          16.8461           10.97m
         3       61801.5990          10.6282           10.92m
         4       61816.3984          12.7198           10.91m
         5       61975.0881          15.7763           11.10m
         6       62162.4589          12.8983           11.08m
         7       61982.1026          12.8914           11.15m
         8       62267.3699          12.4272           10.96m
         9       60894.6215           8.6825           10.80m
        10       62069.7516           9.9121           10.76m
        20       61479.0065           9.3756            9.77m
        30       61627.0932           5.4219            8.96m
        40       61141.8838           4.6263            8.22m
        50    

ValueError: too many values to unpack (expected 8)

In [15]:
df = pd.read_csv('./data/equity-post-HCT-survival-predictions/train.csv')


df['has_hodgekins'] = df['prim_disease_hct'].apply(lambda x: 1 if x == 'HD' else 0)
df['has_hemophagocyticImmuneSyndrome'] = df['prim_disease_hct'].apply(lambda x: 1 if x == 'HIS' else 0)

# df['dri_score'] = df['dri_score'].apply(bin_dri_score)



def categorize_hla_by_percantile(df):
    
    hla_features = ['hla_high_res_8', 'hla_match_a_high', 'hla_match_b_high', 'hla_low_res_6']

    # Function to categorize based on 25th percentile
    def categorize_by_percentile(series):
        threshold = series.quantile(0.25)  # Calculate the 25th percentile
        return np.where(series <= threshold, 0, 1)  # 0 if <= threshold, 1 if above

    # Convert each HLA feature to categorical (0 or 1)
    for feature in hla_features:
        # new_column = f"{feature}_cat"  # Create a new column name for the categorical version
        df[feature] = categorize_by_percentile(df[feature])
        
    return df

df = categorize_hla_by_percantile(df)


    
# Call the function, specifying the ID column to remove
model, preprocessor, X_train, X_val, X_test, y_train, y_val, y_test = split_train_and_evaluate_with_sksurv(
    df,
    train_size=0.7,
    val_size=0.15,
    test_size=0.15,
    id_col='ID'  # Adjust this to match your actual ID column name
)

Dropped column: ID
Training set size: 20160 (70.00%)
Validation set size: 4320 (15.00%)
Test set size: 4320 (15.00%)
      Iter       Train Loss      OOB Improve   Remaining Time 
         1       62295.9481          14.7725           11.03m
         2       62110.7400          16.7796           10.88m
         3       61806.2100          12.1696           10.71m
         4       61828.2801          11.5978           10.58m
         5       61989.1790          15.0982           10.48m
         6       62169.9494          11.3840           10.40m
         7       62000.6606          12.4961           10.32m
         8       62283.0488          11.4669           10.25m
         9       60904.3259          13.3742           10.17m
        10       62081.9527          11.9501           10.09m
        20       61507.7037           8.4885            9.33m
        30       61646.0139           5.4452            8.64m
        40       61164.9362           4.4419            7.90m
        50    

In [5]:
df = pd.read_csv('./data/equity-post-HCT-survival-predictions/train.csv')

df['dri_score'] = df['dri_score'].apply(bin_dri_score)

df['has_hodgekins'] = df['prim_disease_hct'].apply(lambda x: 1 if x == 'HD' else 0)
df['has_hemophagocyticImmuneSyndrome'] = df['prim_disease_hct'].apply(lambda x: 1 if x == 'HIS' else 0)


df['pediatric_and_arrhythmia'] = ((df['dri_score'] == 'N/A - pediatric') & (df['arrhythmia'] == 'Yes')).astype(int)

df = categorize_hla_by_percantile(df)

# Call the function, specifying the ID column to remove
model, preprocessor, X_train, X_val, X_test, y_train, y_val, y_test = split_train_and_evaluate_with_sksurv(
    df,
    train_size=0.7,
    val_size=0.15,
    test_size=0.15,
    id_col='ID'  # Adjust this to match your actual ID column name
)

Dropped column: ID
Training set size: 20160 (70.00%)
Validation set size: 4320 (15.00%)
Test set size: 4320 (15.00%)
      Iter       Train Loss      OOB Improve   Remaining Time 
         1       70358.7919          25.3388           13.98m
         2       69998.0170          24.1502           13.86m
         3       69726.1121          21.8861           13.69m
         4       69891.5344          21.9805           13.57m
         5       70081.8364          19.9264           13.45m
         6       69508.5773          20.0008           13.34m
         7       69415.6389          17.6574           13.24m
         8       70364.2840          16.7081           13.14m
         9       69661.6153          15.9291           13.20m
        10       69502.7763          15.3668           13.21m
        20       69211.9050           7.2304           12.19m
        30       68485.2945           5.9317           11.19m
        40       69326.4869           3.1606           10.25m
        50    

In [10]:
# Load and evaluate the model
df = pd.read_csv('./data/equity-post-HCT-survival-predictions/train.csv')

# Define the mapping for the bins
dri_bins = {
    'High': ['High'],
    'Medium': ['Intermediate', 'High - TED AML case <missing cytogenetics', 
               'Intermediate - TED AML case <missing cytogenetics', 'Low', 
               'Missing disease status'],
    'Low': ['N/A - disease not classifiable', 'N/A - non-malignant indication', 
            'N/A - pediatric', 'TBD cytogenetics', 'Very high']
}

# Function to map dri_score to new bins
def bin_dri_score(score):
    if pd.isna(score):  # Handle NaN values
        return 'Low'  # Assuming NaN goes to 'Low', adjust if needed
    for bin_name, values in dri_bins.items():
        if score in values:
            return bin_name
    return 'Low'  # Default for any unmapped values (e.g., edge cases)


df['dri_score'] = df['dri_score'].apply(bin_dri_score)

df = df.drop(["race_group"], axis=1)

# Call the function, specifying the ID column to remove
model, preprocessor, X_train, X_val, X_test, y_train, y_val, y_test = split_train_and_evaluate_with_sksurv(
    df,
    train_size=0.7,
    val_size=0.15,
    test_size=0.15,
    id_col='ID',  # Adjust this to match your actual ID column name
    categorical_cols = [
        'dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'tbi_status',
        'arrhythmia', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe',
        'prim_disease_hct', 'cmv_status', 'tce_imm_match', 'rituximab',
        'prod_type', 'cyto_score_detail', 'conditioning_intensity', 'ethnicity',
        'obesity', 'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hepatic_severe',
        'prior_tumor', 'peptic_ulcer', 'gvhd_proph', 'rheum_issue', 'sex_match', 'hepatic_mild', 'tce_div_match', 'donor_related',
        'melphalan_dose', 'cardiac', 'pulm_moderate'
    ]
)

Dropped column: ID
Training set size: 20160 (70.00%)
Validation set size: 4320 (15.00%)
Test set size: 4320 (15.00%)
      Iter       Train Loss      OOB Improve   Remaining Time 
         1       70358.8190          25.4356           13.54m
         2       69998.2960          24.1827           13.53m
         3       69726.6986          22.2303           13.43m
         4       69891.9708          21.6603           13.32m
         5       70082.3080          19.7763           13.26m
         6       69508.6662          19.9060           13.16m
         7       69415.7453          17.7379           13.08m
         8       70365.4704          16.4794           12.99m
         9       69662.3624          15.8926           12.90m
        10       69503.8917          15.3288           12.81m
        20       69214.0825           7.2036           11.88m
        30       68487.4745           6.1151           10.95m
        40       69325.7869           3.3005           10.05m
        50    